In [1]:
import os
import requests
import pandas as pd
import io
import csv

In [2]:
path = '/opt/data/GIS-Census/florida'
# filename = path + '/census-post-matching-unmatched.csv'
filename = path + '/soe-test.csv'
# filename = path + '/John_OConeco_original_less_history.csv'
# filename = path + '/SoE-full.csv'

dfDistrict = pd.read_csv(filename, sep=',', engine='python')
dfDistrict

FileNotFoundError: [Errno 2] No such file or directory: '/opt/data/GIS-Census/florida/soe-test.csv'

In [3]:
# import os
# # Get the current working directory
# current_directory = os.getcwd()
# 
# print(current_directory)
# dfDistrict = dfDistrict.dropna(how='any')# 

In [4]:
dfDistrict['City'] =  dfDistrict['City_State'].str.rsplit(' ', n=1).str.get(0)
dfDistrict['State'] = 'FL'

In [5]:
dfDistrict.head()

,VoterID,Voter_Name,Last_Name,First_Name,Middle_Name,Name_Style,Formatted_Address,City_State,Zip_Country,plus4,...,City_Code,City_Ward,Telephone_Number,Voter Status,Sample_Ballots_By_Email,Public_Email_Address,Unnamed: 48,Unnamed: 49,City,State
0,127102014,"Abdullah, Rasheed K",Abdullah,Rasheed,Kaleem,NaN,8442 KONA AVE,JACKSONVILLE FL,32211,NaN,...,0,NaN,(904)982-8385,ACT,N,NaN,NaN,NaN,JACKSONVILLE,FL
1,103605268,"Acosta, Alexandra V",Acosta,Alexandra,V,NaN,7823 Wildwood Rd,Jacksonville FL,32211,NaN,...,0,NaN,NaN,ACT,NaN,NaN,NaN,NaN,Jacksonville,FL
2,103587072,"Acosta, Francisco K",Acosta,Francisco,Klein,NaN,7823 Wildwood Rd,Jacksonville FL,32211,NaN,...,0,NaN,NaN,ACT,NaN,NaN,NaN,NaN,Jacksonville,FL
3,131477247,"Acosta, Juan A",Acosta,Juan,Anibal,NaN,2334 Misty Dr,Jacksonville FL,32211,NaN,...,0,NaN,NaN,ACT,N,NaN,NaN,NaN,Jacksonville,FL
4,116699301,"Acosta, Luis S",Acosta,Luis,S,NaN,1010 Ald Rd,Jacksonville FL,32211,NaN,...,0,NaN,(904)725-8830,ACT,NaN,NaN,NaN,NaN,Jacksonville,FL


In [6]:
# Unique ID, Street address, City, State, ZIP
df_addresses = dfDistrict.filter(items=['VoterID', 'Residence_Address', 'City', 'State', 'Zip_Country'])

In [7]:
df_addresses.head()

,VoterID,Residence_Address,City,State,Zip_Country
0,127102014,1411 Aletha Dr,JACKSONVILLE,FL,32211
1,103605268,7823 Wildwood Rd,Jacksonville,FL,32211
2,103587072,7823 Wildwood Rd,Jacksonville,FL,32211
3,131477247,2334 Misty Dr,Jacksonville,FL,32211
4,116699301,1010 Ald Rd,Jacksonville,FL,32211


In [8]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}

In [9]:
batch_size = 1000
# batch_size = 10
print(f"Batch size: {batch_size}")
dfCensusResults = pd.DataFrame()

Batch size: 1000


In [10]:
for i in range(0, len(df_addresses), batch_size):
    df_address_chunk = df_addresses[i:i + batch_size] 
    address_chunk_name = f"{path}/district_addresses-{i}.csv"
    print(f"{i}) Process batch: ({address_chunk_name}")
    if(os.path.exists(address_chunk_name)):
        # df_address_chunk = pd.read_csv(address_chunk_name, sep=',', header=None)
        print(f"{i}) Source Chunk File exists:{address_chunk_name}")
    else:
        print(f"{i}) Write address chunk:", address_chunk_name)
        df_address_chunk.to_csv(address_chunk_name, index=False, header=False)

    response_chunk_name = f"{path}/district_responses-{i}.csv"
    if(os.path.exists(response_chunk_name)):
        # df_response_chunk = pd.read_csv(response_chunk_name, sep=',', header=None)
        print(f"{i}) RESPONSE chunk File exists:{address_chunk_name}")
    else:
        print(f"{i}) Prepare to write response chunk:{response_chunk_name} from address file: {address_chunk_name}")
        files = {'addressFile': (filename, open(address_chunk_name, 'r'), 'text/csv')}
        s = requests.post(url, files=files, data=payload, timeout=None)
        print("\t\tGot reponse from Census API")
        response_chunk_file = open(response_chunk_name, 'wb')
        response_chunk_file.write(s.content)
        response_chunk_file.close()
        # raw_file = open(f"raw_file-{i}.csv", 'rb+')
        # df_response_chunk = pd.read_csv(raw_file, sep=',', header=None)
    
    # dfCensusResults = dfCensusResults.append(df_response_chunk, ignore_index=True)
    # out_results = f"district-census-{i}.csv"
    # dfCensusResults.to_csv(out_name, index=False)
    

0) Process batch: (/opt/data/GIS-Census/florida/district_addresses-0.csv
0) Write address chunk: /opt/data/GIS-Census/florida/district_addresses-0.csv
0) Prepare to write response chunk:/opt/data/GIS-Census/florida/district_responses-0.csv from address file: /opt/data/GIS-Census/florida/district_addresses-0.csv
		Got reponse from Census API
1000) Process batch: (/opt/data/GIS-Census/florida/district_addresses-1000.csv
1000) Write address chunk: /opt/data/GIS-Census/florida/district_addresses-1000.csv
1000) Prepare to write response chunk:/opt/data/GIS-Census/florida/district_responses-1000.csv from address file: /opt/data/GIS-Census/florida/district_addresses-1000.csv
		Got reponse from Census API
2000) Process batch: (/opt/data/GIS-Census/florida/district_addresses-2000.csv
2000) Write address chunk: /opt/data/GIS-Census/florida/district_addresses-2000.csv
2000) Prepare to write response chunk:/opt/data/GIS-Census/florida/district_responses-2000.csv from address file: /opt/data/GIS-Ce

# Leftover code below...
delete it...?

In [ ]:
dfCensusResults.head()

In [ ]:
dfCensusResults.dtypes

In [ ]:
dfCensusResults.columns = ['id', 'ADDRESS_IN', 'MATCH_INDICATOR', 'MATCH_TYPE', 'ADDRESS_OUT', 'LONG_LAT', 'TIGER_EDGE', 'STREET_SIDE', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK']

In [ ]:
# dfCensus1 = dfCensusResults.loc[dfCensusResults['ADDRESS_IN'].str.contains('95 Jardin')]
# dfCensus1.head()

In [ ]:
hot_dems_file = 'hot-dems.csv'
dfHotDems = pd.read_csv(hot_dems_file, sep=',', quoting=csv.QUOTE_ALL)
dfHotDems.head(10)

In [ ]:
dfHotDems.dtypes

In [ ]:
# dfHotDems1 = dfHotDems.loc[dfHotDems['Address'].str.contains('95 Jardin')]
# dfHotDems1.head(10)

In [ ]:
dfCombined = dfCensusResults.join(dfHotDems, on='id', how='left', lsuffix='_left', rsuffix='_right')

In [ ]:
dfCombined.dtypes

In [ ]:
dfCombined.head()

In [ ]:
# dfCombined1 = dfCensusResults.join(dfHotDems, on='id')

In [ ]:
# dfCombined1.head()

In [ ]:
# dfCombined2 = dfCensusResults.merge(dfHotDems)

In [ ]:
# dfCombined2.head()

In [ ]:
dfCombined.to_csv('combined.csv')